In [1]:
# Ataque de Lagarias al criptosistema de Merkle-Hellman
# Juan Manuel Mateos Pérez

In [2]:
## Explicación :
# En este programa estamos simulando el ataque mediante el método de Lagarías a una comunicación realizada usando el 
# criptosistema de Merkle-Hellman. El ataque utiliza únicamente los valores conocidos del criptosistema, que son la 
# clave pública y el mensaje cifrado. El programa obtiene como resultado un mensaje descifrado, el cual comprobaremos 
# si coincide con el original.

## Ejecución :
# Para ejecutar el programa, solo debemos descomentar el código del main que queramos utilizar:
# (1) Si descomentamos la primera parte, podremos ejecutar el programa 1 vez y veremos todos los datos necesarios. Además, 
# podemos modificar el tamaño del mensaje (variable tam) y el número de iteraciones de la clave privada (variable it).
# (2) Si por otro lado descomentamos la segunda parte, el programa se ejecuta p veces y mostrará un desglose de fallos, 
# vacios y errores de longitud cometidos. En este caso podemos modificar la variable p, que indica la cantidad de cripto-
# sistemas que se van a ejecutar.

In [3]:
# Importamos el método de Merkle-Hellman
load("Merkle-Hellman_iterativo.py")

In [4]:
# ATAQUE DE LAGARIAS

In [5]:
# Genera la matriz necesaria para aplicar Lagarias
# paso == 0 indica generar la matriz inicial, paso != 0 indica generar la matriz del caso 4
def generarMatriz(pk, s, paso):
    n = len(pk)
    filas = []
    
    # generamos los n primeros vectores
    for i in range(0, n):
        aux = zero_vector(n+1)
        aux[i] = 1
        aux[n] = pk[i]*(-1)
        filas.append(aux)

    # generamos el vector n+1
    b = zero_vector(n+1)
    if paso == 0:
        b[n] = s
    else:
        b[n] = sum(pk) - s
    filas.append(b)
    
    return Matrix(filas)

In [6]:
# Encuentra una solución en la matriz recibida
def buscarSolucion(matriz, pk, s):
    n = matriz.nrows()
    sol_encontrada = False
    solucion = []
    
    # compruebo si alguna fila es solución sin aplicar nada
    for j in range(n):
        if sol_encontrada == False:
            suma = 0
            fila = matriz[j]
        
            for i in range(len(fila)-1):
                aux = fila[i]
                if fila[i] < 0:
                    aux = 0
                suma += pk[i]*aux
            if suma == s:
                sol_encontrada = True
                solucion = fila[:-1]
        
    # compruebo si alguna fila es solución tras dividir por un lambda fijo
    if sol_encontrada == False:
        for j in range(n):
            if sol_encontrada == False:
                suma = 0
                fila = matriz[j]
                den  = 1
                den_encontrado = False
                
                for i in range(len(fila)-1):
                    if fila[i] > 0 and den_encontrado == False:
                        den = fila[i]
                        den_encontrado = True
                    aux = fila[i] // den
                    if aux < 0:
                        aux = 0
                    suma += pk[i]*aux
                    solucion.append(aux)
                if suma == s:
                    sol_encontrada = True
                else:
                    solucion = []

    solucion = list(solucion)
    return solucion

In [7]:
# Intercambia los 0 por 1 y viceversa en una matriz
def cambiarMatriz(matriz):
    matriz_res = copy(matriz)

    for i in range(matriz.nrows()):
        for j in range(matriz.ncols()-1):
            if matriz[i, j] == 0:
                matriz_res[i, j] = 1
            elif matriz[i, j] == 1:
                matriz_res[i, j] = 0
    
    return matriz_res

In [8]:
# Aplica el ataque de Lagarias
def ataqueLagarias(pk, s):
    n = len(pk)
    encontrado = False
    solucion = []
    
    # generamos la matriz
    matriz_ini = generarMatriz(pk, s, 0)
    # aplicamos LLL
    matriz_res = matriz_ini.LLL()
    # buscamos una solución
    solucion = buscarSolucion(matriz_res, pk, s)
    
    # cambiamos los 0 por 1 y viceversa
    if len(solucion) == 0:
        matriz_cambio = cambiarMatriz(matriz_res)
        solucion = buscarSolucion(matriz_cambio, pk, s)
    
    # caso 4 del algoritmo
    if len(solucion) == 0:
        matriz_ini = generarMatriz(pk, s, 1)
        matriz_res = matriz_ini.LLL() 
        solucion = buscarSolucion(matriz_res, pk, s)
        
        # cambiamos los 0 por 1 y viceversa
        if len(solucion) == 0:
            matriz_cambio = cambiarMatriz(matriz_res)
            solucion = buscarSolucion(matriz_cambio, pk, s)
        
    return solucion

In [9]:
# Calcula el número de errores cometidos
def comprobarErrores(men_orig, men_obt):
    n = len(men_orig)
    vector_dif = []

    if len(men_obt) == 0:
        return len(men_orig)

    for i in range(0, n):
        vector_dif.append(abs(men_orig[i] - men_obt[i]))

    return sum(vector_dif)

In [10]:
# DATOS DE SALIDA

In [11]:
# ejecuta y muestra los datos tras aplicar una iteración
def unaIteracion(tam, it):
    merkle_hellman = Merkle_Hellman(tam, it)
    merkle_hellman.do()

    pk = merkle_hellman.pk
    s  = merkle_hellman.s

    lagarias = ataqueLagarias(pk, s)

    print()
    print("Clave pública      :", merkle_hellman.pk)
    print("Mensaje original   :", merkle_hellman.mensaje)
    print("Mensaje cifrado    :", merkle_hellman.s)
    print("Mensaje descifrado :", lagarias)
    print("Tamaño mensaje     :", tam)
    print("Número iteraciones :", it)
    print("Errores totales    :", comprobarErrores(merkle_hellman.mensaje, lagarias))

In [12]:
# ejecuta y muestra los datos tras aplicar n iteraciones
def variasIteraciones(n):
    errores  = 0
    err_long = 0
    vacios   = 0
    densidad = 0

    print("Iteración \t Tamaño vector \t Número Iteraciones \t Densidad \t\tResultado")
    for i in range(p):
        print(i+1, end = "")

        tam = random.randint(3, 100)
        print("\t\t", tam, end="")

        it = random.randint(0, 3)
        print("\t\t", it, end="")

        merkle_hellman = Merkle_Hellman(tam, it)
        merkle_hellman.do()

        densidad = RR(merkle_hellman.tamano / log(max(merkle_hellman.pk), 2))
        print("\t\t\t", densidad, end="")

        lagarias = ataqueLagarias(merkle_hellman.pk, merkle_hellman.s)
        if len(lagarias) == 0:
            vacios += 1
            print("\tvacio")
        elif len(lagarias) != len(merkle_hellman.mensaje):
            err_long += 1
            print("\terror longitud")
        else:
            valor = comprobarErrores(merkle_hellman.mensaje, lagarias)
            errores += valor
            if valor != 0:
                print("\terror valor")
            else:
                print("\tobtenido")

    print("\nErrores totales  tras", p, "iteraciones :", errores)
    print("Vacios  totales  tras", p, "iteraciones :", vacios)
    print("Errores longitud tras", p, "iteraciones :", err_long)

In [13]:
# MAIN

In [14]:
print("Ataque Lagarias")

# ---------- descomentar para realizar 1 ejecución aleatoria ----------
# tam = random.randint(3, 100)
# it  = random.randint(0, 3)
# unaIteracion(tam, it)

# ---------- descomentar para realizar p ejecuciones aleatorias ----------
# p = 10
# variasIteraciones(p)

Ataque Lagarias
